In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# **데이터 살펴보기**

In [1]:
import pandas as pd
import warnings 
warnings.filterwarnings(action='ignore')
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import re

In [ ]:
#경로 설정
import os
os.chdir('/content/drive/My Drive/월간 09/Writing_Style/Writing_Style')

In [2]:
#파일 불러오기
train = pd.read_csv('train.csv', encoding = 'utf-8')
test = pd.read_csv('test_x.csv', encoding = 'utf-8')
sample_submission = pd.read_csv('sample_submission.csv', encoding = 'utf-8')

In [3]:
#train 데이터 살펴보기
train

,index,text,author
0,0,"He was almost choking. There was so much, so m...",3
1,1,"“Your sister asked for it, I suppose?”",2
2,2,"She was engaged one day as she walked, in per...",1
3,3,"The captain was in the porch, keeping himself ...",4
4,4,"“Have mercy, gentlemen!” odin flung up his han...",3
...,...,...,...
54874,54874,"“Is that you, Mr. Smith?” odin whispered. “I h...",2
54875,54875,"I told my plan to the captain, and between us ...",4
54876,54876,"""Your sincere well-wisher, friend, and sister...",1
54877,54877,“Then you wanted me to lend you money?”,3


In [4]:
#test 데이터 살펴보기
test

,index,text
0,0,“Not at all. I think she is one of the most ch...
1,1,"""No,"" replied he, with sudden consciousness, ""..."
2,2,As the lady had stated her intention of scream...
3,3,“And then suddenly in the silence I heard a so...
4,4,His conviction remained unchanged. So far as I...
...,...,...
19612,19612,"At the end of another day or two, odin growing..."
19613,19613,"All afternoon we sat together, mostly in silen..."
19614,19614,"odin, having carried his thanks to odin, proc..."
19615,19615,"Soon after this, upon odin's leaving the room,..."


In [5]:
#sample_submission
sample_submission

,index,0,1,2,3,4
0,0,0,0,0,0,0
1,1,0,0,0,0,0
2,2,0,0,0,0,0
3,3,0,0,0,0,0
4,4,0,0,0,0,0
...,...,...,...,...,...,...
19612,19612,0,0,0,0,0
19613,19613,0,0,0,0,0
19614,19614,0,0,0,0,0
19615,19615,0,0,0,0,0


# **전처리**

In [6]:
#부호를 제거해주는 함수
def alpha_num(text):
    return re.sub(r'[^A-Za-z0-9 ]', '', text)

train['text']=train['text'].apply(alpha_num)

In [7]:
#부호가 사라진 것을 확인할 수 있습니다.
train

,index,text,author
0,0,He was almost choking There was so much so muc...,3
1,1,Your sister asked for it I suppose,2
2,2,She was engaged one day as she walked in peru...,1
3,3,The captain was in the porch keeping himself c...,4
4,4,Have mercy gentlemen odin flung up his hands D...,3
...,...,...,...
54874,54874,Is that you Mr Smith odin whispered I hardly d...,2
54875,54875,I told my plan to the captain and between us w...,4
54876,54876,Your sincere wellwisher friend and sister LUC...,1
54877,54877,Then you wanted me to lend you money,3


In [8]:
# 불용어 제거해주는 함수
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stopwords:
            final_text.append(i.strip())
    return " ".join(final_text)

# 불용어
stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", 
             "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", 
             "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", 
             "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", 
             "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", 
             "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", 
             "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", 
             "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", 
             "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", 
             "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", 
             "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]

In [9]:
#전처리 적용
train['text'] = train['text'].str.lower()
test['text'] = test['text'].str.lower()
train['text'] = train['text'].apply(alpha_num).apply(remove_stopwords)
test['text'] = test['text'].apply(alpha_num).apply(remove_stopwords)

In [10]:
# train test 분리
X_train = np.array([x for x in train['text']])
X_test = np.array([x for x in test['text']])
y_train = np.array([x for x in train['author']])


In [25]:
X_train.shape

(54879,)

# **모델링**

In [12]:
#파라미터 설정
vocab_size = 20000
embedding_dim = 16
max_length = 500
padding_type='post'
#oov_tok = "<OOV>"

In [13]:
#tokenizer에 fit
tokenizer = Tokenizer(num_words = vocab_size)#, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

In [14]:
#데이터를 sequence로 변환해주고 padding 해줍니다.
train_sequences = tokenizer.texts_to_sequences(X_train)
train_padded = pad_sequences(train_sequences, padding=padding_type, maxlen=max_length)

test_sequences = tokenizer.texts_to_sequences(X_test)
test_padded = pad_sequences(test_sequences, padding=padding_type, maxlen=max_length)

In [15]:
#가벼운 NLP모델 생성
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

In [16]:
# compile model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
print(model.summary())


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 16)           320000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 24)                408       
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 125       
Total params: 320,533
Trainable params: 320,533
Non-trainable params: 0
_________________________________________________________________
None


In [17]:
# fit model
num_epochs = 20
history = model.fit(train_padded, y_train, 
                    epochs=num_epochs, verbose=2, 
                    validation_split=0.2)

Epoch 1/20
1372/1372 - 3s - loss: 1.5609 - accuracy: 0.2781 - val_loss: 1.5258 - val_accuracy: 0.2828
Epoch 2/20
1372/1372 - 3s - loss: 1.4468 - accuracy: 0.3564 - val_loss: 1.3661 - val_accuracy: 0.4472
Epoch 3/20
1372/1372 - 2s - loss: 1.2604 - accuracy: 0.5036 - val_loss: 1.1941 - val_accuracy: 0.5418
Epoch 4/20
1372/1372 - 2s - loss: 1.0908 - accuracy: 0.5735 - val_loss: 1.0668 - val_accuracy: 0.5866
Epoch 5/20
1372/1372 - 3s - loss: 0.9855 - accuracy: 0.6160 - val_loss: 1.0027 - val_accuracy: 0.6077
Epoch 6/20
1372/1372 - 3s - loss: 0.9137 - accuracy: 0.6485 - val_loss: 0.9519 - val_accuracy: 0.6303
Epoch 7/20
1372/1372 - 3s - loss: 0.8565 - accuracy: 0.6746 - val_loss: 0.9178 - val_accuracy: 0.6451
Epoch 8/20
1372/1372 - 2s - loss: 0.8071 - accuracy: 0.6954 - val_loss: 0.9223 - val_accuracy: 0.6454
Epoch 9/20
1372/1372 - 3s - loss: 0.7578 - accuracy: 0.7189 - val_loss: 0.8660 - val_accuracy: 0.6701
Epoch 10/20
1372/1372 - 3s - loss: 0.7168 - accuracy: 0.7354 - val_loss: 0.8714 - 

In [18]:
# predict values
pred = model.predict_proba(test_padded)

Instructions for updating:
Please use `model.predict()` instead.


In [19]:
pred

array([[3.3930733e-04, 4.4000819e-01, 9.8530881e-02, 4.6053848e-01,
        5.8312388e-04],
       [9.1948405e-02, 7.4418145e-01, 6.3933678e-02, 7.1830889e-03,
        9.2753470e-02],
       [9.9861777e-01, 9.3963841e-04, 1.1392005e-08, 4.5406141e-09,
        4.4264246e-04],
       ...,
       [5.5225607e-04, 9.9943560e-01, 6.6276598e-09, 1.2140148e-05,
        4.3546180e-08],
       [5.3348165e-04, 9.9945492e-01, 6.9833398e-08, 1.1139291e-05,
        3.7966041e-07],
       [9.8079282e-01, 2.8634877e-05, 9.0404810e-06, 7.6028169e-07,
        1.9168783e-02]], dtype=float32)

In [20]:
# submission
sample_submission[['0','1','2','3','4']] = pred
sample_submission

,index,0,1,2,3,4
0,0,0.000339,4.400082e-01,9.853088e-02,4.605385e-01,5.831239e-04
1,1,0.091948,7.441815e-01,6.393368e-02,7.183089e-03,9.275347e-02
2,2,0.998618,9.396384e-04,1.139200e-08,4.540614e-09,4.426425e-04
3,3,0.000020,1.333243e-08,9.187879e-01,2.961874e-05,8.116236e-02
4,4,0.859225,3.861887e-02,1.448801e-02,4.206795e-02,4.560036e-02
...,...,...,...,...,...,...
19612,19612,0.000002,9.999983e-01,3.176585e-21,1.828560e-12,7.953081e-18
19613,19613,0.058718,4.063752e-05,5.157047e-05,2.525718e-09,9.411896e-01
19614,19614,0.000552,9.994356e-01,6.627660e-09,1.214015e-05,4.354618e-08
19615,19615,0.000533,9.994549e-01,6.983340e-08,1.113929e-05,3.796604e-07


In [21]:
sample_submission.to_csv('submission.csv', index = False, encoding = 'utf-8')